<img src="particle_filter_finished.gif">

In [ ]:
import sys
import pdb
import random
import scipy
import scipy.stats
import scipy.io
import numpy
import math
import matplotlib
import matplotlib.pyplot as plt
from lab8_common import *
from copy import deepcopy
from numpy import matrix
from scipy.stats import multivariate_normal

import Figtodat
from images2gif import writeGif

%matplotlib inline

In [ ]:
number = 0
def is_running():
    global number 
    number = number + 1
    print('\r' + str(number), end="")
    return

In [ ]:
class Particle(object):
    
    
    
    def __init__(self):
        self.weight = 0.0
        return
    
    
    
    def update_prediction(self):
        return
    
    
    
    def update_weight(self, new_measurement):
        return
    
    
    

# Proposal Distribution

My proposal distribution is the uniform distribution which is implemented 
by random.random(). This distribution simulates that the robot 
thinks he might be in any room.

    function particle_init():
    
        x_position = random.random() 
        y_position = random.random() 
        orientation = random.random() * 2 * math.pi
  


# Sensor Likelihood
My sensor likelihood is implemented by the sum of squares error between the robot's ray measurements, and the 
the individual particle's simulated ray measurements

    function update_weight(observation_measurement):
    
        prediction_measurement = cast_rays(this_particle_position)
        
        total_error = 0.0
        for i in observation_measurement:
            local_error = observation_measurement[i] - prediction_measurement[i] 
            total_error = total_error + (local_error * local_error)
            
        weight = 1 / total_error
        

In [ ]:
class RobotParticle(Particle):
    
    
    
    def __init__(self, room_map = None):
        self.weight = 0.0
        self.weight_distribution = 0.0
        
        self.room_map = room_map
        self.x_position = random.random() 
        self.y_position = random.random() 
        self.orientation = random.random() * 2 * math.pi
        return  
    
    
    
    def move_particle(self):
        walk = ((random.random() - 0.5) * 0.2)
        turn = ((random.random() - 0.5) * 0.5) 
        self.orientation = (self.orientation + turn) % (2 * math.pi)
        walk_x_forward = self.x_position + (walk * math.sin(self.orientation))
        walk_x_backward = self.x_position - (walk * math.sin(self.orientation))
        walk_y_forward = self.y_position + (walk * math.cos(self.orientation))
        walk_y_backward = self.y_position - (walk * math.cos(self.orientation))
        self.x_position = walk_x_forward if walk_x_forward < 1 and walk_x_forward > 0 else walk_x_backward 
        self.y_position = walk_y_forward if walk_y_forward < 1 and walk_y_forward > 0 else walk_y_backward   
        return
    
    
    
    def non_normalized_gaussian_pdf(self, x, mean, sd):
        var = float(sd)**2
        pi = 3.1415926
        #denom = (2*pi*var)**.5
        num = math.exp(-(float(x)-float(mean))**2/(2*var))
        return num #num/denom
    
    
    
    def non_normalized_multivariate_gaussian_pdf(self, x, mu, sigma):
        size = len(x)
        if size == len(mu) and (size, size) == sigma.shape:
            det = numpy.linalg.det(sigma)
            if det == 0:
                raise NameError("The covariance matrix can't be singular")
            #norm_const = 1.0/ ( math.pow((2*pi),float(size)/2) * math.pow(det,1.0/2) )
            x_mu = numpy.matrix(x - mu)
            inv = sigma.I        
            result = math.pow(math.e, -0.5 * (x_mu * inv * x_mu.T))
            return result #norm_const * result
        else:
            raise NameError("The dimensions of the input don't match")
            
            
    
    def update_non_normalized_weight(self, observation_measurement):
        
        this_particle_position = numpy.array([[self.x_position],
                                              [self.y_position],
                                              [self.orientation]])
        
        prediction_measurement = cast_rays(this_particle_position, self.room_map).ravel()

        total_error = 0.0
        for i in range(len(observation_measurement)):
            local_error = (observation_measurement[i] - prediction_measurement[i]) 
            total_error = total_error + (local_error * local_error)
        self.weight = 1 / total_error

        return self.weight

# Resampling

By using the uniform distribution implemented by random.random(), I sample from the distribution of particles' weights implemented by a list of weights. 
(In my code, it is a list of particles that have a weight_distribution property)

    function resample_particles():
    
        new_particles = list()
        
        for i in particles:
            point = random.random() 
            for particle in particles:
                if point is inside particle.weight_distribution:
                    new_particles.append(deepcopy(particle))
                    break
                    
        particles = new_particles
       

In [ ]:
class ParticleFilter(object):
    
    
    
    def __init__(self, particles):
        self.particles = particles
        return

    
    
    def update_particles(self, measurement):
        self.move_particles()
        non_normalized_weight_distribution_length = self.update_weights(measurement)
        self.resample_particles(non_normalized_weight_distribution_length)
        return self.particles
    
    
    
    def move_particles(self):
        for particle in self.particles:
            particle.move_particle()
        return
    
    
    
    def update_weights(self, measurement):
        weight_sum_so_far = 0.0
        
        for particle in self.particles:
            weight_sum_so_far = weight_sum_so_far + particle.update_non_normalized_weight(measurement)
            particle.weight_distribution = weight_sum_so_far
        
        return weight_sum_so_far
    
    
    
    def resample_particles(self, non_normalized_weight_distribution_length):
        resampled_particles = list()
        for counter in range(len(self.particles)):
            random_point = random.random() * non_normalized_weight_distribution_length
            for particle in self.particles:
                if random_point < particle.weight_distribution:
                    resampled_particles.append(deepcopy(particle))
                    break
        self.particles = resampled_particles
        return

                    
                    
    def get_particle_max_weight(self):
        max_particle = RobotParticle()
        for particle in self.particles:
            if max_particle.weight < particle.weight:
                max_particle = particle
        return max_particle
              
        
            

# Expected Value

    function expected_value():
    
        length = len(particles)
        
        x_mean = 0
        y_mean = 0
        
        for particle in particles:
            x_mean = x_mean + particle.x_position
            y_mean = y_mean + particle.y_position
            
        x_mean = x_mean / length
        y_mean = y_mean / length
        
        return x_mean, y_mean

In [ ]:
class Main(object):
    
    
    
    def __init__(self):
        self.room_map = create_map()
        particles = [RobotParticle(self.room_map) for i in range(600)]
        sensors = scipy.io.loadmat('sensors.mat')
        self.true_positions = sensors['true_states'].T
        self.measurements = sensors['sonars'].T
        self.particle_filter = ParticleFilter(particles)
        return
        
        
    def run(self):
        images = list()
        for time in range(len(self.measurements)):
            measurement = self.measurements[time]
            true_position = self.true_positions[time]
            particles = self.particle_filter.update_particles(measurement)
            figure = self.display(true_position, particles)
            image = Figtodat.fig2img(figure)
            images.append(image)
            is_running()
            
        writeGif("particleFilter.gif",images,duration=0.10,dither=0)
        return
      
        
        
    def display(self, true_position, particles):
        plt.close("all")
        figure = plt.figure()
        #figure.clf()
        true_position = numpy.array([[true_position[0]],
                                     [true_position[1]],
                                     [true_position[2]]])
        show_map(self.room_map)
        cast_rays(true_position, self.room_map, verbose=True)
        x_positions, y_positions, x_mean, y_mean = self.extract_positions(particles)
        plt.scatter(x_positions, y_positions, color='blue', linewidth=1, alpha=0.5, label="particles")
        plt.scatter(x_mean, y_mean, color='red', linewidth=4, alpha=1, label="mean")
        
        figure.suptitle("Particle Filter", fontsize=20)
        plt.legend(loc='upper right')#,prop={'size':20}, bbox_to_anchor=(1, 0.5))
        #figure.set_size_inches(3, 2)
        
        return figure
    
    
    
    def extract_positions(self, particles):
        length = len(particles)
        
        x_positions = list()
        y_positions = list()
        
        x_mean = 0
        y_mean = 0
        
        for particle in particles:
            x_mean = x_mean + particle.x_position
            y_mean = y_mean + particle.y_position
            x_positions.append(particle.x_position)
            y_positions.append(particle.y_position)
            
        x_mean = x_mean / length
        y_mean = y_mean / length
        
        return x_positions, y_positions, x_mean, y_mean
    
                            
            
            

In [ ]:
main = Main()
main.run()